<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_6_25_model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#데이터셋 URL
url =  'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

#칼럼명 정의 (데이터셋에 헤더가 없음)
columns = [
 'age', 'workclass', 'fnlwgt', 'education', 'education-num',
 'marital-status', 'occupation', 'relationship', 'race', 'sex',
 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
 ]

# na_values 파라미터를 사용하여 '?'를 결측값으로 처리하며 데이터 로드
df = pd.read_csv(url, header=None, names=columns, na_values='?', skipinitialspace=True)

# 데이터의 첫 5행 확인
print(df.head())

In [ ]:
print("---데이터 정보---")
df.info()

# 수치형 데이터의 기술 통계량 확인
print("\n--- 수치형 데이터 기술 통계량 ---")
print(df.describe())

# 결측값 확인
print("\n---컬럼별 결측값 개수---")
print(df.isnull().sum())

In [ ]:
# income 변수 분포 확인
sns.countplot(x='income', data=df)
plt.title('Income Distribution')
plt.show()

# 비율 확인
income_counts = df['income'].value_counts(normalize=True)
print(income_counts)

In [ ]:
# 수치형 특성 리스트
numerical_features = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']

# 각 수치형 특성에 대한 히스토그램과 박스 플롯 시각화
for feature in numerical_features:
    plt.figure(figsize=(12, 5))

    # 히스토그램
    plt.subplot(1, 2, 1)
    sns.histplot(data=df, x=feature, kde=True)
    plt.title(f'Histogram of {feature}')

    # 박스 플롯
    plt.subplot(1, 2, 2)
    sns.boxplot(data=df, x='income', y=feature)
    plt.title(f'Box Plot of {feature} by Income')

    plt.tight_layout()
    plt.show()

In [ ]:
# 범주형 특성 리스트
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']

# 각 범주형 특성에 따른 소득 분포 시각화
for feature in categorical_features:
 plt.figure(figsize=(12, 6))
 sns.countplot(data=df, y=feature, hue='income', order=df[feature].value_counts().index)
 plt.title(f'Income Distribution by {feature}')
 plt.tight_layout()
 plt.show()

In [ ]:
# 결측값이 있는 범주형 컬럼에 대해 최빈값으로 대체
for col in ['workclass', 'occupation', 'native-country']:
    df[col].fillna(df[col].mode(), inplace=True)

# 결측값 처리 확인
print("\n---컬럼별 결측값 개수 (결측값 처리 후)---")
print(df.isnull().sum())

In [ ]:
# 불필요하거나 중복되는 컬럼 제거
df.drop(['education', 'fnlwgt'], axis=1, inplace=True)

In [ ]:
# 목표 변수(Income)를 0과 1로 변환
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

# 범주형 특성을 원-핫 인코딩으로 변환
df_encoded = pd.get_dummies(df, drop_first=True)

#변환된 데이터 확인
print(df_encoded.head())

In [ ]:
importances = rf_clf.feature_importances_

feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,   # 82개의 컬럼
    'importance': importances     # 각 피처의 중요도 값
}).sort_values('importance', ascending=False)

top_n = 20
selected_features = feature_importance_df['feature'].head(top_n).tolist()
print("중요한 상위 20개 피처:")
print(selected_features)


In [ ]:
from sklearn.preprocessing import StandardScaler

# 특성과 타겟 분리
X = df_encoded.drop('income', axis=1)
y = df_encoded['income']

# 스케일러 객체 생성
scaler = StandardScaler()

# 데이터에 스케일러 적용
X_scaled = scaler.fit_transform(X)

# 스케일링된 데이터를 다시 데이터프레임으로 변환 (칼럼명 유지를 위해)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print(X_scaled.describe())

In [ ]:
from sklearn.model_selection import train_test_split

# 학습 데이터와 테스트 데이터로 분할 (7:3 비율, 클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

print("학습 데이터 크기:", X_train.shape)
print("테스트 데이터 크기:", X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

#로지스틱 회귀 모델 객체 생성 및 학습
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# 생성된 데이터에 대한 예측
y_pred_lr = log_reg.predict(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 결정 트리 모델 객체 생성 및 학습
# max_depth를 설정하여 과적합을 방지할 수 있음 (여기서는 기본값 사용)
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred_dt = dt_clf.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 객체 생성 및 학습
# n_estimators는 생성할 트리의 개수를 의미함
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

#  각 모델에 대한 혼동 행렬 계산
cm_lr = confusion_matrix(y_test, y_pred_lr)
cm_dt = confusion_matrix(y_test, y_pred_dt)
cm_rf = confusion_matrix(y_test, y_pred_rf)

# 혼동 행렬 시각화 (예시: 랜덤 포레스트)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix for Random Forest')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))  # 1행 3열

# Logistic Regression
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_title('Confusion Matrix for Logistic Regression')
axes[0].set_xlabel('Predicted Labels')
axes[0].set_ylabel('True Labels')

# Decision Tree
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[1])
axes[1].set_title('Confusion Matrix for Decision Trees')
axes[1].set_xlabel('Predicted Labels')
axes[1].set_ylabel('')

# Random Forest
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[2])
axes[2].set_title('Confusion Matrix for Random Forest')
axes[2].set_xlabel('Predicted Labels')
axes[2].set_ylabel('')  # y label 생략

plt.tight_layout()
plt.show()

In [ ]:
#로지스틱 회귀 분류 리포트
print("--- Logistic Regression Classification Report ---")
print(classification_report(y_test, y_pred_lr))

# 결정 트리 분류 리포트
print("\n--- Decision Tree Classification Report ---")
print(classification_report(y_test, y_pred_dt))

# 랜덤 포레스트 분류 리포트
print("\n--- Random Forest Classification Report ---")
print(classification_report(y_test, y_pred_rf))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 각 모델의 예측 확률 계산 (Positive 클래스에 대한 확률)
prob_lr = log_reg.predict_proba(X_test)[:, 1]
prob_dt = dt_clf.predict_proba(X_test)[:, 1]
prob_rf = rf_clf.predict_proba(X_test)[:, 1]

# ROC 곡선 계산
fpr_lr, tpr_lr, _ = roc_curve(y_test, prob_lr)
fpr_dt, tpr_dt, _ = roc_curve(y_test, prob_dt)
fpr_rf, tpr_rf, _ = roc_curve(y_test, prob_rf)

# AUC 계산
auc_lr = auc(fpr_lr, tpr_lr)
auc_dt = auc(fpr_dt, tpr_dt)
auc_rf = auc(fpr_rf, tpr_rf)

# ROC 곡선 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.2f})')
plt.plot(fpr_dt, tpr_dt, label=f'Decision Tree (AUC = {auc_dt:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.2f})')

# Random Chance (대각선)
plt.plot([0, 1], [0, 1], 'k--', label='Random Chance')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.tree import export_graphviz
import graphviz

# 결정 트리 시각화를 위한 DOT 데이터 생성 (트리 깊이를 3으로 제한하여 가독성 확보)
# class_names에 특수 문자가 포함되어 있어 오류가 발생할 수 있으므로, 유효한 문자열로 대체.
dot_data = export_graphviz(dt_clf,
                           max_depth=3,
                           out_file=None,
                           feature_names=X_train.columns,
                           class_names=['less_than_50K', 'greater_than_50K'], # 특수 문자 제거
                           filled=True,
                           rounded=True,
                           special_characters=True)

# Graphviz를 사용하여 시각화
graph = graphviz.Source(dot_data)
graph.render("adult_decision_tree") # PDF 파일로 저장
graph

In [ ]:
# 랜덤 포레스트 모델로부터 특성 중요도 추출
# 100개의 트리에서 평균적으로 어떤 변수가 중요한지 보고 싶음.

importances = rf_clf.feature_importances_

# 특성 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': importances
}).sort_values('importance', ascending=False)

# 특성 중요도 시각화
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(15)) # 상위 15개만 표시
plt.title('Top 15 Feature Importances from Random Forest')
plt.show()

# 특성 중요도 표
print(feature_importance_df)

### 향후 연구 방향
- 하이퍼파라미터 튜닝
- 다른 알고리즘 탐색:XGBoost, LightGBM과 같은 그래디언트 부스팅(Gradient Boosting) 계열의 알고리즘이나 서포트 벡터 머신(Support Vector Machine, SVM) 등 다른 강력한 분류 알고리즘을 적용하여 성능을 비교
- 피드백 로프 구축: 모델 해석 → 특성 공학 → 모델 재학습 → 재평가의 반복적인 과정


In [ ]:
# 랜덤 포레스트 특성 중요도 결과 사용 (feature_importance_df는 cell 5KcY3vndqytI에서 생성됨)
# 상위 20개 특성 선택
top_n = 20
selected_features = feature_importance_df['feature'].head(top_n).tolist()

print(f"선택된 상위 {top_n}개 특성:")
print(selected_features)

# 선택된 특성들로 학습 및 테스트 데이터셋 구성
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

print("\n선택된 특성으로 구성된 학습 데이터 크기:", X_train_selected.shape)
print("선택된 특성으로 구성된 테스트 데이터 크기:", X_test_selected.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 로지스틱 회귀 모델 재학습
log_reg_selected = LogisticRegression(random_state=42)
log_reg_selected.fit(X_train_selected, y_train)

# 결정 트리 모델 재학습
dt_clf_selected = DecisionTreeClassifier(random_state=42)
dt_clf_selected.fit(X_train_selected, y_train)

# 랜덤 포레스트 모델 재학습
rf_clf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf_selected.fit(X_train_selected, y_train)

print("모델 재학습 완료.")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 재학습된 모델로 예측 수행
y_pred_lr_selected = log_reg_selected.predict(X_test_selected)
y_pred_dt_selected = dt_clf_selected.predict(X_test_selected)
y_pred_rf_selected = rf_clf_selected.predict(X_test_selected)

# 각 모델에 대한 혼동 행렬 계산
cm_lr_selected = confusion_matrix(y_test, y_pred_lr_selected)
cm_dt_selected = confusion_matrix(y_test, y_pred_dt_selected)
cm_rf_selected = confusion_matrix(y_test, y_pred_rf_selected)

# 각 모델에 대한 분류 리포트 출력
print("--- Logistic Regression (Selected Features) Classification Report ---")
print(classification_report(y_test, y_pred_lr_selected))

print("\n--- Decision Tree (Selected Features) Classification Report ---")
print(classification_report(y_test, y_pred_dt_selected))

print("\n--- Random Forest (Selected Features) Classification Report ---")
print(classification_report(y_test, y_pred_rf_selected))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming confusion matrices cm_lr_selected, cm_dt_selected, and cm_rf_selected have been calculated

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))  # 1행 3열

# Logistic Regression (Selected Features)
sns.heatmap(cm_lr_selected, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0])
axes[0].set_title('Confusion Matrix for Logistic Regression (Selected Features)')
axes[0].set_xlabel('Predicted Labels')
axes[0].set_ylabel('True Labels')

# Decision Tree (Selected Features)
sns.heatmap(cm_dt_selected, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[1])
axes[1].set_title('Confusion Matrix for Decision Trees (Selected Features)')
axes[1].set_xlabel('Predicted Labels')
axes[1].set_ylabel('')  # y label 생략

# Random Forest (Selected Features)
sns.heatmap(cm_rf_selected, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[2])
axes[2].set_title('Confusion Matrix for Random Forest (Selected Features)')
axes[2].set_xlabel('Predicted Labels')
axes[2].set_ylabel('')  # y label 생략

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 1. 모델 생성
rf = RandomForestClassifier(random_state=42)

# 2. 파라미터 공간 축소
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],  # 'auto'는 deprecated
    'max_depth': [10, 30, 50, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True]  # False 제외 (대부분 True 사용): True면 각 tree를 학습시킬 때 최대 max_samples로 전달한 숫자만큼의 데이터 셋을 구축한다. False면 모든 decision tree를 만들 때 전체 데이터를 사용
}

# 3. RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,  # 20개의 조합만 샘플링
    cv=3,       # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1   # 병렬 처리
)

# 4. 모델 튜닝 수행
random_search.fit(X_train_selected, y_train)

# 5. 결과 출력
print("최적 하이퍼파라미터:", random_search.best_params_)
print("최적 모델 성능 (교차 검증 평균 정확도):", random_search.best_score_)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# 학습된 데이터셋 기준
# X_train_selected, y_train → 학습 데이터
# X_test_selected, y_test → 테스트 데이터
# 여기선 테스트 데이터로 평가

# 모델들 생성
logistic = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier(random_state=42)
tuned_rf = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='log2',
    max_depth=50,
    bootstrap=True,
    random_state=42
)

# 학습
logistic.fit(X_train_selected, y_train)
decision_tree.fit(X_train_selected, y_train)
tuned_rf.fit(X_train_selected, y_train)

# 예측
y_pred_log = logistic.predict(X_test_selected)
y_pred_tree = decision_tree.predict(X_test_selected)
y_pred_rf = tuned_rf.predict(X_test_selected)

# 혼동 행렬
cm_log = confusion_matrix(y_test, y_pred_log)
cm_tree = confusion_matrix(y_test, y_pred_tree)
cm_rf = confusion_matrix(y_test, y_pred_rf)

# 시각화
plt.figure(figsize=(18, 5))

models = ['Logistic Regression', 'Decision Trees', 'Random Forest (Tuned)']
cms = [cm_log, cm_tree, cm_rf]

for i, cm in enumerate(cms):
    plt.subplot(1, 3, i+1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix for {models[i]}')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# 튜닝된 랜덤 포레스트 모델의 분류 리포트 출력
print("--- Tuned Random Forest Classification Report ---")
print(classification_report(y_test, y_pred_rf))

RandomForestClassifier를 활용해 랜덤 포레스트의 하이퍼파라미터를 튜닝.
트리 개수(n_estimators), 최대 깊이(max_depth), 분할 조건(min_samples_split, min_samples_leaf) 등 총 6개의 파라미터로 탐색했고,
각 조합에 대해 교차 검증을 수행하여 가장 성능이 좋은 설정을 선택함.
그 결과 모델의 정확도를 약 82%에서 86%로 향상.
특히 양성클래스에 대한 예측 성능(f1-score)도 함께 개선함

param_dist = {
    'n_estimators': [100, 200, 300],        # 트리 개수
    'max_depth': [30, 50, None],           # 트리 최대 깊이
    'min_samples_split': [2, 5],           # 분할 최소 샘플 수
    'min_samples_leaf': [1, 2],            # 리프 노드 최소 샘플 수
    'max_features': ['sqrt', 'log2'],      # 분할 시 사용할 특성 수
    'bootstrap': [True]                    # 복원 추출 여부
}




In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 모델 정의
logistic = LogisticRegression(max_iter=1000)
tree = DecisionTreeClassifier(random_state=42)
tuned_rf = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='log2',
    max_depth=50,
    bootstrap=True,
    random_state=42
)

# 학습
logistic.fit(X_train_selected, y_train)
tree.fit(X_train_selected, y_train)
tuned_rf.fit(X_train_selected, y_train)

# 예측 확률
y_score_log = logistic.predict_proba(X_test_selected)[:, 1]
y_score_tree = tree.predict_proba(X_test_selected)[:, 1]
y_score_rf = tuned_rf.predict_proba(X_test_selected)[:, 1]

# ROC Curve
fpr_log, tpr_log, _ = roc_curve(y_test, y_score_log)
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_score_tree)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_score_rf)

# AUC 계산
auc_log = auc(fpr_log, tpr_log)
auc_tree = auc(fpr_tree, tpr_tree)
auc_rf = auc(fpr_rf, tpr_rf)

# 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr_log, tpr_log, label=f"Tuned Logistic Regression (AUC = {auc_log:.2f})")
plt.plot(fpr_tree, tpr_tree, label=f"Tuned Decision Tree (AUC = {auc_tree:.2f})")
plt.plot(fpr_rf, tpr_rf, label=f"Tuned Random Forest (AUC = {auc_rf:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label='Random Chance')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Tuned Model ROC Curve Comparison")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()
